In [17]:
from supabase import create_client, Client
from dotenv import dotenv_values
import pandas as pd
from ast import literal_eval

In [2]:
env = dotenv_values(".env")
url: str = env["URL"]
key: str = env["KEY"]

supabase: Client = create_client(url, key)

In [12]:
response = supabase.table("provas").select("*").execute()

2023-11-28 00:39:30,844:INFO - HTTP Request: GET https://zxdvzjonlwyzotqugand.supabase.co/rest/v1/provas?select=%2A "HTTP/1.1 200 OK"


In [18]:
df = pd.DataFrame(response.data)

In [19]:
df.alternativas = df.alternativas.apply(literal_eval)

In [23]:
df.alternativas = df.alternativas.apply(lambda l: " ".join(l))

In [24]:
df.head()

,id,questao,enunciado,alternativas,gabarito
0,1,1.0,Sentimos que toda satisfação de nossos desejos...,consagração de relacionamentos afetivos. admi...,B
1,2,2.0,Batizado por Tancredo Neves de “Nova República...,representação do legislativo com a fórmula do ...,C
2,3,3.0,"Os moradores de Andalsnes, na Noruega, poderia...",Êxodo rural. Movimento pendular. Migração de...,B
3,4,4.0,O Rio de Janeiro tem projeção imediata no próp...,Frente pioneira. Zona de transição. Região p...,C
4,5,5.0,TEXTO I Documentos do século XVI algumas vezes...,"concepção idealizada do território, entendido ...",C


In [25]:
df.enunciado = df.enunciado + df.alternativas

In [32]:
df.drop(columns=['alternativas'], inplace=True)

In [40]:
df.dropna(inplace=True)

In [41]:
df.head()

,id,questao,enunciado,gabarito
0,1,1.0,Sentimos que toda satisfação de nossos desejos...,B
1,2,2.0,Batizado por Tancredo Neves de “Nova República...,C
2,3,3.0,"Os moradores de Andalsnes, na Noruega, poderia...",B
3,4,4.0,O Rio de Janeiro tem projeção imediata no próp...,C
4,5,5.0,TEXTO I Documentos do século XVI algumas vezes...,C


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [63]:
stop_words = []
with open("stopwords", "r", encoding="utf-8") as file:
    while True:
        word = file.readline()
        if not word:
            break
        stop_words.append(word.strip())
print(stop_words)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estav

In [64]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(df.enunciado)

In [65]:
true_k = 7
model = KMeans(n_clusters=true_k, init="k-means++", max_iter=100, n_init=1)
model.fit(X)

KMeans(max_iter=100, n_clusters=7, n_init=1)

In [66]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for i in range(true_k):
    print("Cluster %d:" % i, end=' ')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end=' ')
    print()

Top terms per cluster:
Cluster 0:  profundidade  trabalho  adaptado  redes  sociais  áreas  texto  polpa  social  acesso 
Cluster 1:  g3  g68  g72  g82  g86  g76  g85  g71  g81  g87 
Cluster 2:  sobre  texto  tempo  adaptado  acesso  novo  disponível  terra  todos  paulo 
Cluster 3:  the  to  000  of  and  is  células  in  it  with 
Cluster 4:  meio  rio  dia  texto  adaptado  página  janeiro  caderno  azul  ch 
Cluster 5:  água  10  cm  100  cada  temperatura  50  número  maior  tipo 
Cluster 6:  la  texto  en  el  acesso  disponível  sobre  dia  arte  www 
